In [2]:
% load_ext autoreload
% autoreload 2

In [3]:
import utils
import train
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

In [26]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 20

num_epochs=10

In [27]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()

/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [ ]:
train.train(model, model_name, train_iter, val_iter, DE, EN, num_epochs, gpu)


  0%|          | 0/3722 [00:00<?, ?it/s]/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "

  4%|▎         | 136/3722 [00:09<04:20, 13.75it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_gr

Epoch: 1 ValPerpBound: 167.2720 NLLBound: 64.5816 RE: 64.5547 KL: 0.0269


 39%|███▉      | 1447/3722 [01:32<02:24, 15.72it/s]

In [ ]:
import utils
val_perp = utils.perp_bound(model, val_iter, gpu)
print(val_perp)

In [ ]:
EN.vocab.stoi['</s>']

In [7]:
import beam_search
import torch
k = 10
bos = EN.vocab.stoi['<s>']
eos = EN.vocab.stoi['</s>']
max_len = 20
    
for batch in train_iter:
    if gpu:
        src = batch.src.cuda()
    else:
        src = batch.src
    break


src = src[:,:1].contiguous()
print(src.size())

torch.Size([8, 1])


In [ ]:
src = src.cuda()
print(src)

In [ ]:
beam_search.beam_search(k, max_len, model, src, bos, eos, gpu)

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm

In [9]:
if gpu: 
    src = src.cuda()

# run encoder
encoded_src = model.encode(src) # batch size = 1

# init 
init_prob = -1e10
best_options = [(init_prob, [bos], None)] # beam

In [23]:
for length in range(max_len): # maximum target length
    options = [] # candidates 

    for lprob, sentence, hidden in best_options:
        # Prepare last word
        last_word = sentence[-1]

        # keep sentences ending in '</s>' as candidates
        if last_word == eos:
            options.append((lprob, sentence, current_state))

        else:
            last_word_input = torch.tensor([last_word], requires_grad=False).long().view(1,1)
            if gpu: last_word_input = last_word_input.cuda()

            # Decode
            lprobs, new_hidden = model.generate(last_word_input, encoded_src, hidden)
            # Add top k candidates to options list for next word

    lprobs = lprobs.squeeze()
    for index in torch.topk(lprobs, k)[1]:         
        option = (lprobs[index].item() + lprob, sentence + [index], new_hidden) 
        options.append(option)

    options.sort(key = lambda x: x[0], reverse=True) # sort by lprob
    best_options = options[:k] # place top candidates in beam

In [25]:
for lprob, sentence, _ in best_options:
    string = ""
    for s in sentence:
        string += EN.vocab.itos[s] + " "
    print("lprob: {:.4f} sentence: \n {}".format(lprob, string))

lprob: -10000000718.4467 sentence: 
 <s> compassion relatively eating leaf We innovate : account maybe gifted them put gave . manage accessories or dream compassion see join your think dream story reader picked affect manage this . The closet compassion : your , compassion righting drive shaping Russia Paper join see created . pieces policy . compassion manage pieces figure reader can product account , this The adversity . so I That Russia trip innovate figure dream policy The Russia see , your give created <pad> 
lprob: -10000000718.9462 sentence: 
 <s> compassion relatively eating leaf We innovate : account maybe gifted them put gave . manage accessories or dream compassion see join your think dream story reader picked affect manage this . The closet compassion : your , compassion righting drive shaping Russia Paper join see created . pieces policy . compassion manage pieces figure reader can product account , this The adversity . so I That Russia trip innovate figure dream policy Th

In [22]:
l1 = ["hello", "my", "name", "is"]
l2 = ["hello", "my", "as", "ad"]

In [25]:
import utils
print(utils.bleu(l1, l2))
print(utils.rouge(l1, l2))

0.6389431042462724
0.4999999950000001


/home/artidoro/.local/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
